<a href="https://colab.research.google.com/github/rohittaware1997/NLPTeam24/blob/main/Task4_Team24_vr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#author: Vatsal Verma and Rohit Taware
!pip install transformers==4.9.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 1.1 MB/s 
     |████████████████████████████████| 3.3 MB 62.2 MB/s 
     |████████████████████████████████| 880 kB 54.1 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=ad3744163f6253e147e6d0a2e84c63839b6e23dcb13cca68462f2e3e0254656d
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses


In [ ]:
!pip install datasets==1.10.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 542 kB 30.8 MB/s 
     |████████████████████████████████| 132 kB 79.9 MB/s 
     |████████████████████████████████| 212 kB 75.9 MB/s 


# Export dataset

In [ ]:
import traceback
import csv

import pandas as pd


def write_tsv_dataframe(filepath, dataframe):
    """
        Stores `DataFrame` as tsv file
        Parameters
        ----------
        filepath : str
            Path to tsv file
        dataframe : pd.DataFrame
            DataFrame to store
        Raises
        ------
        IOError
            if the file can't be opened
    """
    try:
        dataframe.to_csv(filepath, encoding='utf-8', sep='\t', index=False, header=True, quoting=csv.QUOTE_NONE)
    except IOError:
        traceback.print_exc()

# Format Dataset

In [ ]:
import pandas as pd


def combine_columns(df_arguments, df_labels):
    """Combines the two `DataFrames` on column `Argument ID`"""
    return pd.merge(df_arguments, df_labels, on='Argument ID')


def split_arguments(df_arguments):
    """Splits `DataFrame` by column `Usage` into `train`-, `validation`-, and `test`-arguments"""
    train_arguments = df_arguments.loc[df_arguments['Usage'] == 'train'].drop(['Usage'], axis=1).reset_index(drop=True)
    valid_arguments = df_arguments.loc[df_arguments['Usage'] == 'validation'].drop(['Usage'], axis=1).reset_index(drop=True)
    test_arguments = df_arguments.loc[df_arguments['Usage'] == 'test'].drop(['Usage'], axis=1).reset_index(drop=True)
    
    return train_arguments, valid_arguments, test_arguments


def create_dataframe_head(argument_ids, model_name):
    """
        Creates `DataFrame` usable to append predictions to it
        Parameters
        ----------
        argument_ids : list[str]
            First column of the resulting DataFrame
        model_name : str
            Second column of DataFrame will contain the given model name
        Returns
        -------
        pd.DataFrame
            prepared DataFrame
    """
    df_model_head = pd.DataFrame(argument_ids, columns=['Argument ID'])
    df_model_head['Method'] = [model_name] * len(argument_ids)

    return df_model_head

# Import Dataset

In [ ]:
import traceback
import pandas as pd
import json


class MissingColumnError(AttributeError):
    """Error indicating that an imported DataFrame lacks necessary columns"""
    pass


def load_json_file(filepath):
    """Load content of json-file from `filepath`"""
    with open(filepath, 'r') as  json_file:
        return json.load(json_file)


def load_values_from_json(filepath):
    """Load values per level from json-file from `filepath`"""
    json_values = load_json_file(filepath)
    values = { "1":set(), "2":set(), "3":set(), "4a":set(), "4b":set() }
    for value in json_values["values"]:
        values["1"].add(value["name"])
        values["2"].add(value["level2"])
        for valueLevel3 in value["level3"]:
            values["3"].add(valueLevel3)
        for valueLevel4a in value["level4a"]:
            values["4a"].add(valueLevel4a)
        for valueLevel4b in value["level4b"]:
            values["4b"].add(valueLevel4b)
    values["1"] = sorted(values["1"])
    values["2"] = sorted(values["2"])
    values["3"] = sorted(values["3"])
    values["4a"] = sorted(values["4a"])
    values["4b"] = sorted(values["4b"])
    return values


def load_arguments_from_tsv(filepath, default_usage='test'):
    """
        Reads arguments from tsv file
        Parameters
        ----------
        filepath : str
            The path to the tsv file
        default_usage : str, optional
            The default value if the column "Usage" is missing
        Returns
        -------
        pd.DataFrame
            the DataFrame with all arguments
        Raises
        ------
        MissingColumnError
            if the required columns "Argument ID" or "Premise" are missing in the read data
        IOError
            if the file can't be read
        """
    try:
        dataframe = pd.read_csv(filepath, encoding='utf-8', sep='\t', header=0)
        if not {'Argument ID', 'Premise'}.issubset(set(dataframe.columns.values)):
            raise MissingColumnError('The argument "%s" file does not contain the minimum required columns [Argument ID, Premise].' % filepath)
        if 'Usage' not in dataframe.columns.values:
            dataframe['Usage'] = [default_usage] * len(dataframe)
        return dataframe
    except IOError:
        traceback.print_exc()
        raise


def load_labels_from_tsv(filepath, label_order):
    """
        Reads label annotations from tsv file
        Parameters
        ----------
        filepath : str
            The path to the tsv file
        label_order : list[str]
            The listing and order of the labels to use from the read data
        Returns
        -------
        pd.DataFrame
            the DataFrame with the annotations
        Raises
        ------
        MissingColumnError
            if the required columns "Argument ID" or names from `label_order` are missing in the read data
        IOError
            if the file can't be read
        """
    try:
        dataframe = pd.read_csv(filepath, encoding='utf-8', sep='\t', header=0)
        dataframe = dataframe[['Argument ID'] + label_order]
        return dataframe
    except IOError:
        traceback.print_exc()
        raise
    except KeyError:
        raise MissingColumnError('The file "%s" does not contain the required columns for its level.' % filepath)

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 30.6 MB/s 


# BERT Models (can experiment with different models, and parameter like batch_size and learning rate)

In [ ]:
import torch
from datasets import (Dataset, DatasetDict, load_dataset)
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, PreTrainedModel, BertModel, BertForSequenceClassification, TrainingArguments, Trainer)
from sklearn.metrics import f1_score
# from transformers import MBartTokenizer
from transformers import DistilBertTokenizer, DistilBertModel, DistilBertTokenizerFast
from transformers import RobertaTokenizer
from transformers import LongformerTokenizer
from transformers import BertTokenizer
import numpy as np


def accuracy_thresh(y_pred, y_true, thresh=0.5, sigmoid=True):
    """Compute accuracy of predictions"""
    y_pred = torch.from_numpy(y_pred)
    y_true = torch.from_numpy(y_true)
    if sigmoid:
        y_pred = y_pred.sigmoid()

    return ((y_pred > thresh) == y_true.bool()).float().mean().item()


def f1_score_per_label(y_pred, y_true, value_classes, thresh=0.5, sigmoid=True):
    """Compute label-wise and averaged F1-scores"""
    y_pred = torch.from_numpy(y_pred)
    y_true = torch.from_numpy(y_true)
    if sigmoid:
        y_pred = y_pred.sigmoid()

    y_true = y_true.bool().numpy()
    y_pred = (y_pred > thresh).numpy()

    f1_scores = {}
    for i, v in enumerate(value_classes):
        f1_scores[v] = round(f1_score(y_true[:, i], y_pred[:, i], zero_division=0), 2)

    f1_scores['avg-f1-score'] = round(np.mean(list(f1_scores.values())), 2)

    return f1_scores


def compute_metrics(eval_pred, value_classes):
    """Custom metric calculation function for MultiLabelTrainer"""
    predictions, labels = eval_pred
    f1scores = f1_score_per_label(predictions, labels, value_classes)
    return {'accuracy_thresh': accuracy_thresh(predictions, labels), 'f1-score': f1scores,
            'marco-avg-f1score': f1scores['avg-f1-score']}


class MultiLabelTrainer(Trainer):
    """
        A transformers `Trainer` with custom loss computation
        Methods
        -------
        compute_loss(model, inputs, return_outputs=False):
            Overrides loss computation from Trainer class
        """
    def compute_loss(self, model, inputs, return_outputs=False):
        """Custom loss computation"""
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.BCEWithLogitsLoss()
        loss = loss_fct(logits.view(-1, self.model.config.num_labels),
                        labels.float().view(-1, self.model.config.num_labels))
        return (loss, outputs) if return_outputs else loss


def tokenize_and_encode(examples):
    """Tokenizes each arguments "Premise" """
    return tokenizer(examples['Premise'], truncation=True)


def convert_to_dataset(train_dataframe, test_dataframe, labels):
    """
        Converts pandas DataFrames into a DatasetDict
        Parameters
        ----------
        train_dataframe : pd.DataFrame
            Arguments to be listed as "train"
        test_dataframe : pd.DataFrame
            Arguments to be listed as "test"
        labels : list[str]
            The labels in both DataFrames
        Returns
        -------
        tuple(DatasetDict, list[str])
            a `DatasetDict` with attributes "train" and "test" for the listed arguments,
            a `list` with the contained labels
        """
    column_intersect = [x for x in (['Premise'] + labels) if x in train_dataframe.columns.values]

    train_dataset = Dataset.from_dict((train_dataframe[column_intersect]).to_dict('list'))
    test_dataset = Dataset.from_dict((test_dataframe[column_intersect]).to_dict('list'))

    ds = DatasetDict()
    ds['train'] = train_dataset
    ds['test'] = test_dataset

    ds = ds.map(lambda x: {"labels": [int(x[c]) for c in ds['train'].column_names if
                                      c not in ['Argument ID', 'Conclusion', 'Stance', 'Premise', 'Part']]})

    cols = ds['train'].column_names
    cols.remove('labels')

    ds_enc = ds.map(tokenize_and_encode, batched=True, remove_columns=cols)

    cols.remove('Premise')

    return ds_enc, cols

#For BERT
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

#For DistilBERT
# tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# For RoBERTa
# tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

#For BERT-tiny
# tokenizer = BertTokenizer.from_pretrained("prajjwal1/bert-tiny")

#For BERT-small
# tokenizer = BertTokenizer.from_pretrained("prajjwal1/bert-small")

#For BERT-medium
# tokenizer = BertTokenizer.from_pretrained("prajjwal1/bert-medium")


def load_model_from_data_dir(model_dir, num_labels):
    """Loads Bert model from specified directory and converts to CUDA model if available"""
    model = AutoModelForSequenceClassification.from_pretrained(model_dir, num_labels=num_labels)
    if torch.cuda.is_available():
        return model.to('cuda')
    return model


def predict_bert_model(dataframe, model_dir, labels):
    """
        Classifies each argument using the Bert model stored in `model_dir`
        Parameters
        ----------
        dataframe: pd.Dataframe
            The arguments to be classified
        model_dir: str
            The directory of the pre-trained Bert model to use
        labels: list[str]
            The labels to predict
        Returns
        -------
        np.ndarray
            numpy nd-array with the predictions given by the model
        """
    ds, no_labels = convert_to_dataset(dataframe, dataframe, labels)
    num_labels = len(labels)
    ds = ds.remove_columns(['labels'])

    batch_size = 8
    args = TrainingArguments(
        output_dir=model_dir,
        do_train=False,
        do_eval=False,
        do_predict=True,
        per_device_eval_batch_size=batch_size
    )

    model = load_model_from_data_dir(model_dir, num_labels=num_labels)

    multi_trainer = MultiLabelTrainer(
        model,
        args,
        tokenizer=tokenizer
    )

    prediction = 1 * (multi_trainer.predict(ds['train']).predictions > 0.5)

    return prediction


def train_bert_model(train_dataframe, model_dir, labels, test_dataframe=None, num_train_epochs=20):
    """
        Trains Bert model with the arguments in `train_dataframe`
        Parameters
        ----------
        train_dataframe: pd.DataFrame
            The arguments to be trained on
        model_dir: str
            The directory for storing the trained model
        labels : list[str]
            The labels in the training data
        test_dataframe: pd.DataFrame, optional
            The validation arguments (default is None)
        num_train_epochs: int, optional
            The number of training epochs (default is 20)
        Returns
        -------
        Metrics
            result of validation if `test_dataframe` is not None
        NoneType
            otherwise
        """
    if test_dataframe is None:
        test_dataframe = train_dataframe
    ds, labels = convert_to_dataset(train_dataframe, test_dataframe, labels)

    batch_size = 8

    args = TrainingArguments(
        output_dir=model_dir,
        evaluation_strategy="steps",
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_train_epochs,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model='marco-avg-f1score'
    )

    #Load the required model with appropriate name as defined above 
    model = load_model_from_data_dir("bert-base-uncased", num_labels=len(labels))

    multi_trainer = MultiLabelTrainer(
        model,
        args,
        train_dataset=ds["train"],
        eval_dataset=ds["test"],
        compute_metrics=lambda x: compute_metrics(x, labels),
        tokenizer=tokenizer
    )

    multi_trainer.train()

    model.save_pretrained(model_dir)

    if test_dataframe is not None:
        return multi_trainer.evaluate()

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

# 1-Baseline

In [ ]:
import numpy as np
import pandas as pd


def predict_one_baseline(dataframe, labels):
    """
        Classifies each argument in the dataframe as corresponding to all labels.
        Parameters
        ----------
        dataframe : pd.DataFrame
            The arguments to be classified
        labels : list[str]
            The listing of all labels
        Returns
        -------
        DataFrame
            the predictions given by the model
        """
    return pd.DataFrame(np.full((len(dataframe), len(labels)), 1, dtype=int), columns=labels)

# SVM Model where we can change parameters C and max_iter

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score

import pandas as pd
import numpy as np

import json

# constant label values
vocab_label = 'vocabulary'
idf_label = 'idf'
intercept_label = 'intercept'
coef_label = 'coef'


class MyLinearSVC(LinearSVC):
    """
        A class to load and store a pretrained linear svm for predictions
        ...
        Attributes
        ----------
        intercept : int
            The intercept constant
        coef : ndarray
            The coefficients for all observed features
        Methods
        -------
        predict(X):
            Overrides `predict(X)` from LinearSVC
    """

    def __init__(self, intercept, coef):
        """
            Constructs all necessary attributes for the MyLinearSVC object
            Parameters
            ----------
            intercept : int
                The intercept constant
            coef : list[int]
                The coefficients for all observed features
        """
        LinearSVC.__init__(self, C=18, class_weight='balanced', max_iter=20000)
        self.intercept = intercept
        self.coef = np.asarray(coef)
        self.size = len(coef)

    def predict(self, X):
        """
            Predict class labels for samples in X
            Parameters
            ----------
            X : {array-like, sparse matrix} of shape (n_samples, n_features)
                The data matrix for which we want to get the predictions
            Returns
            -------
            ndarray of shape (n_samples,)
                Vector containing the class labels for each sample
        """
        input = np.transpose(X.todense())
        matrix = np.squeeze(np.asarray(self.__my_predict(input)))
        return np.vectorize(lambda x: 1 if x >= 0.5 else 0)(matrix)

    def __my_predict(self, input):
        """
            Predict class label probability for samples in input
            Parameters
            ----------
            input : ndarray of shape (n_features, n_samples)
                The data matrix for which we want to get the predictions
            Returns
            -------
            ndarray of shape (n_samples,)
                Vector containing the class labels for each sample
        """
        result = input[0] * self.coef[0]
        for i in range(1, self.size):
            result += input[i] * self.coef[i]
        result += self.intercept
        return result


def predict_svm(dataframe, labels, vectorizer_file, model_file):
    """
        Classifies each argument in the dataframe using the trained Support Vector Machines (SVMs) in the `model_file`
        Parameters
        ----------
        dataframe : pd.DataFrame
            The arguments to be classified
        labels : list[str]
            The listing of all labels
        vectorizer_file : str
            The file containing the fitted data from the TfidfVectorizer
        model_file : str
            The file containing the serialized SVM models
        Returns
        -------
        DataFrame
            the predictions given by the model
        """
    input_vector = dataframe['Premise']
    df_model_predictions = {}

    # load vectorizer
    with open(vectorizer_file, "r") as f:
        vectorizer_json = json.load(f)

    vocabulary = vectorizer_json[vocab_label]
    idf = np.asarray(vectorizer_json[idf_label])

    vectorizer = TfidfVectorizer(vocabulary=vocabulary)
    # vectorizer = CountVectorizer(vocabulary=vocabulary)
    # vectors = vectorizer.fit_transform(corpus)
    vectorizer.idf_ = idf

    with open(model_file, "r") as f:
        model_json = json.load(f)

    for label_name in labels:
        model_dict = model_json[label_name]
        svm = Pipeline([
            ('tfidf', vectorizer),
            ('clf', MyLinearSVC(intercept=model_dict[intercept_label], coef=model_dict[coef_label])),
        ])
        df_model_predictions[label_name] = svm.predict(input_vector)

    return pd.DataFrame(df_model_predictions, columns=labels)


def train_svm(train_dataframe, labels, vectorizer_file, model_file, test_dataframe=None):
    """
        Trains Support Vector Machines (SVMs) on the arguments in the train_dataframe and saves them in `model_file`
        Parameters
        ----------
        train_dataframe : pd.DataFrame
            The arguments to be trained on
        labels : list[str]
            The listing of all labels
        vectorizer_file : str
            The file for storing the fitted data from the TfidfVectorizer
        model_file : str
            The file for storing the serialized SVM models
        test_dataframe : pd.DataFrame, optional
            The validation arguments (default is None)
        Returns
        -------
        dict
            f1-scores of validation if `test_dataframe` is not None
        NoneType
            otherwise
        """
    train_input_vector = train_dataframe['Premise']
    if test_dataframe is not None:
        valid_input_vector = test_dataframe['Premise']
        f1_scores = {}

    # vectorizer = CountVectorizer(stop_words='english')
    vectorizer = TfidfVectorizer(stop_words='english')
    vectorizer.fit(train_input_vector)

    vectorizer_json = {vocab_label: vectorizer.vocabulary_, idf_label: vectorizer.idf_.tolist()}

    with open(vectorizer_file, "w") as f:
        json.dump(vectorizer_json, f)

    # dictionary for storing model data
    model_json = {}

    for label_name in labels:
        svm = Pipeline([
            ('tfidf', vectorizer),
            ('clf', OneVsRestClassifier(LinearSVC(C=18, class_weight='balanced', max_iter=20000), n_jobs=1)),
        ])
        svm.fit(train_input_vector, train_dataframe[label_name])

        coef = np.squeeze(np.asarray(svm.steps[1][1].estimators_[0].coef_)).tolist()
        intercept = svm.steps[1][1].estimators_[0].intercept_[0]
        model_json[label_name] = {intercept_label: intercept, coef_label: coef}

        if test_dataframe is not None:
            valid_pred = svm.predict(valid_input_vector)
            f1_scores[label_name] = round(f1_score(test_dataframe[label_name], valid_pred, zero_division=0), 2)

    with open(model_file, "w") as f:
        json.dump(model_json, f)

    if test_dataframe is not None:
        f1_scores['avg-f1-score'] = round(np.mean(list(f1_scores.values())), 2)
        return f1_scores

# Training

In [ ]:
import sys
import getopt
import os

# from components.setup import (load_values_from_json, load_arguments_from_tsv, load_labels_from_tsv,
#                                                 combine_columns, split_arguments)
# from components.models import (train_bert_model, train_svm)

help_string = '\nUsage:  training.py [OPTIONS]' \
              '\n' \
              '\nTrain the BERT model (and optional SVM) on the arguments' \
              '\n' \
              '\nOptions:' \
              '\n  -c, --classifier string  Select classifier: "b" for Bert, "s" for SVM, "bs" for both (default' \
              '\n                           "b")' \
              '\n  -d, --data-dir string    Directory with the argument files (default "/data/")' \
              '\n  -h, --help               Display help text' \
              '\n  -l, --levels string      Comma-separated list of taxonomy levels to train models for (default' \
              '\n                           "1,2,3,4a,4b")' \
              '\n  -m, --model-dir string   Directory for saving the trained models (default "/models/")' \
              '\n  -v, --validate           Request evaluation after training'


def main():
    # default values
    curr_dir = os.getcwd()
    run_bert = False
    run_svm = True
    data_dir = '/content/data_dir'
    levels = ["1", "2", "3", "4a", "4b"]
    model_dir = '/content/models/'
    validate = True

    # try:
    #     opts, args = getopt.gnu_getopt(argv, "c:d:hl:m:v", ["classifier=", "data-dir=", "help", "levels=", "model-dir=", "validate"])
    # except getopt.GetoptError:
    #     print(help_string)
    #     sys.exit(2)
    # for opt, arg in opts:
    #     if opt in ('-h', '--help'):
    #         print(help_string)
    #         sys.exit()
    #     elif opt in ('-c', '--classifier'):
    #         run_bert = 'b' in arg.lower()
    #         run_svm = 's' in arg.lower()
    #         if not run_bert and not run_svm:
    #             print('No classifiers selected')
    #             sys.exit(2)
    #     elif opt in ('-d', '--data-dir'):
    #         data_dir = arg
    #     elif opt in ('-l', '--levels'):
    #         levels = arg.split(",")
    #     elif opt in ('-m', '--model-dir'):
    #         model_dir = arg
    #     elif opt in ('-v', '--validate'):
    #         validate = True

    svm_dir = os.path.join(model_dir, 'svm')

    # Check data directory
    if not os.path.isdir(data_dir):
        print('The specified data directory "%s" does not exist' % data_dir)
        sys.exit(2)

    # Check model directory
    if os.path.isfile(model_dir):
        print('The specified <model-dir> "%s" points to an existing file' % model_dir)
        sys.exit(2)
    if os.path.isdir(model_dir) and len(os.listdir(model_dir)) > 0:
        print('The specified <model-dir> "%s" already exists and contains files' % model_dir)
        decision = input('Do You still want to proceed? [y/n]\n').lower()
        if decision != 'y':
            sys.exit(-1)
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)

    if run_svm and not os.path.isdir(svm_dir):
        if os.path.exists(svm_dir):
            print('Unable to create svm directory at "%s"' % svm_dir)
        else:
            os.mkdir(svm_dir)

    argument_filepath = os.path.join(data_dir, 'arguments.tsv')
    value_json_filepath = os.path.join(data_dir, 'values.json')

    if not os.path.isfile(argument_filepath):
        print('The required file "arguments.tsv" is not present in the data directory')
        sys.exit(2)
    if not os.path.isfile(value_json_filepath):
        print('The required file "values.json" is not present in the data directory')
        sys.exit(2)

    # load arguments
    df_arguments = load_arguments_from_tsv(argument_filepath, default_usage='train')
    if len(df_arguments) < 1:
        print('There are no arguments in file "%s"' % argument_filepath)
        sys.exit(2)

    values = load_values_from_json(value_json_filepath)
    num_levels = len(levels)

    # check levels
    for i in range(num_levels):
        if levels[i] not in values:
            print('Missing attribute "{}" in value.json'.format(levels[i]))
            sys.exit(2)

    # format dataset
    df_train_all = []
    df_valid_all = []
    for i in range(num_levels):
        label_filepath = os.path.join(data_dir, 'labels-level{}.tsv'.format(levels[i]))
        if not os.path.isfile(label_filepath):
            print('The required file "labels-level{}.tsv" is not present in the data directory'.format(levels[i]))
            sys.exit(2)
        # read labels from .tsv file
        df_labels = load_labels_from_tsv(label_filepath, values[levels[i]])
        # join arguments and labels
        df_full_level = combine_columns(df_arguments, df_labels)
        # split dataframe by usage
        train_arguments, valid_arguments, _ = split_arguments(df_full_level)
        df_train_all.append(train_arguments)
        df_valid_all.append(valid_arguments)

    if len(df_train_all[0]) < 1:
        print('There are no arguments listed for training.')
        sys.exit()

    if validate and len(df_valid_all[0]) < 1:
        print('There are no arguments listed for validation. Proceeding without validation.')
        validate = False

    # train bert model
    if run_bert:
        for i in range(num_levels):
            print("===> Bert: Training Level %s..." % levels[i])
            if validate:
                bert_model_evaluation = train_bert_model(df_train_all[i],
                                                         os.path.join(model_dir, 'bert_train_level{}'.format(levels[i])),
                                                         values[levels[i]], test_dataframe=df_valid_all[i])
                print("F1-Scores for Level %s:" % levels[i])
                print(bert_model_evaluation['eval_f1-score'])
            else:
                train_bert_model(df_train_all[i], os.path.join(model_dir, 'bert_train_level{}'.format(levels[i])),
                                 values[levels[i]])

    if run_svm:
        for i in range(num_levels):
            print("===> SVM: Training Level %s..." % levels[i])
            if validate:
                svm_f1_scores = train_svm(df_train_all[i], values[levels[i]],
                                          os.path.join(model_dir, 'svm/svm_train_level{}_vectorizer.json'.format(levels[i])),
                                          os.path.join(model_dir, 'svm/svm_train_level{}_models.json'.format(levels[i])),
                                          test_dataframe=df_valid_all[i])
                print("F1-Scores for Level %s:" % levels[i])
                print(svm_f1_scores)
            else:
                train_svm(df_train_all[i], values[levels[i]],
                          os.path.join(model_dir, 'svm/svm_train_level{}_vectorizer.json'.format(levels[i])),
                          os.path.join(model_dir, 'svm/svm_train_level{}_models.json'.format(levels[i])))


if __name__ == '__main__':
    main()

The specified <model-dir> "/content/models/" already exists and contains files
Do You still want to proceed? [y/n]
y
===> SVM: Training Level 1...
F1-Scores for Level 1:
{'Be ambitious': 0.32, 'Be behaving properly': 0.11, 'Be broadminded': 0.18, 'Be capable': 0.28, 'Be choosing own goals': 0.35, 'Be compliant': 0.12, 'Be courageous': 0.0, 'Be creative': 0.0, 'Be curious': 0.19, 'Be daring': 0.0, 'Be forgiving': 0.0, 'Be helpful': 0.22, 'Be holding religious faith': 0.45, 'Be honest': 0.13, 'Be honoring elders': 0.29, 'Be humble': 0.0, 'Be independent': 0.06, 'Be intellectual': 0.46, 'Be just': 0.36, 'Be logical': 0.17, 'Be loving': 0.0, 'Be neat and tidy': 0.0, 'Be polite': 0.0, 'Be protecting the environment': 0.0, 'Be respecting traditions': 0.11, 'Be responsible': 0.25, 'Be self-disciplined': 0.0, 'Have a comfortable life': 0.3, 'Have a good reputation': 0.0, 'Have a safe country': 0.68, 'Have a sense of belonging': 0.0, 'Have a stable society': 0.27, 'Have a varied life': 0.11, 'H

# Prediction

In [ ]:
import sys
import getopt
import os
import pandas as pd

# from components.setup import (load_values_from_json, load_arguments_from_tsv, split_arguments,
#                               write_tsv_dataframe, create_dataframe_head)
# from components.models import (predict_bert_model, predict_one_baseline, predict_svm)

help_string = '\nUsage:  predict.py [OPTIONS]' \
              '\n' \
              '\nRequest prediction of the BERT model (and optional SVM / 1-Baseline) for all test arguments' \
              '\n' \
              '\nOptions:' \
              '\n  -c, --classifier string  Select classifier: "b" for Bert, "s" for SVM, "o" for 1-Baseline,' \
              '\n                           or combination like "so" (default "b")' \
              '\n  -d, --data-dir string    Directory with the argument files (default "/data/")' \
              '\n  -h, --help               Display help text' \
              '\n  -l, --levels string      Comma-separated list of taxonomy levels to train models for (default' \
              '\n                           "1,2,3,4a,4b")' \
              '\n  -m, --model-dir string   Directory for saving the trained models (default "/models/")' \
              '\n  -o, --output-dir string  Directory to write the "predictions.tsv" into (default "/output/")'


def main():
    # default values
    curr_dir = os.getcwd()
    run_bert = False
    run_svm = True
    run_one_baseline = False
    data_dir = '/content/data_dir'
    levels =["1", "2", "3", "4a", "4b"]
    model_dir = '/content/models/'
    output_dir = '/content/output/'

    # try:
    #     opts, args = getopt.gnu_getopt(argv, "c:d:hl:m:o:",
    #                                    ["classifier=", "data-dir=", "help", "levels=", "model-dir=", "output-dir="])
    # except getopt.GetoptError:
    #     print(help_string)
    #     sys.exit(2)
    # for opt, arg in opts:
    #     if opt in ('-h', '--help'):
    #         print(help_string)
    #         sys.exit()
    #     elif opt in ('-c', '--classifier'):
    #         run_bert = 'b' in arg.lower()
    #         run_svm = 's' in arg.lower()
    #         run_one_baseline = 'o' in arg.lower()
    #         if not run_bert and not run_svm and not run_one_baseline:
    #             print('No classifiers selected')
    #             sys.exit(2)
    #     elif opt in ('-d', '--data-dir'):
    #         data_dir = arg
    #     elif opt in ('-l', '--levels'):
    #         levels = arg.split(",")
    #     elif opt in ('-m', '--model-dir'):
    #         model_dir = arg
    #     elif opt in ('-o', '--output-dir'):
    #         output_dir = arg

    # Check data directory
    if not os.path.isdir(data_dir):
        print('The specified data directory "%s" does not exist' % data_dir)
        sys.exit(2)

    argument_filepath = os.path.join(data_dir, 'arguments.tsv')
    values_filepath = os.path.join(data_dir, 'values.json')

    if not os.path.isfile(argument_filepath):
        print('The required file "arguments.tsv" is not present in the data directory')
        sys.exit(2)
    if not os.path.isfile(values_filepath):
        print('The required file "values.json" is not present in the data directory')
        sys.exit(2)

    # load arguments
    df_arguments = load_arguments_from_tsv(argument_filepath)
    if len(df_arguments) < 1:
        print('There are no arguments in file "%s"' % argument_filepath)
        sys.exit(2)

    values = load_values_from_json(values_filepath)
    num_levels = len(levels)

    # check levels
    for i in range(num_levels):
        if levels[i] not in values:
            print('Missing attribute "{}" in value.json'.format(levels[i]))
            sys.exit(2)

    # check model directory
    if not os.path.isdir(model_dir):
        print('The specified <model-dir> "%s" does not exist' % model_dir)
        sys.exit(2)

    for i in range(num_levels):
        if run_bert and not os.path.exists(os.path.join(model_dir, 'bert_train_level{}'.format(levels[i]))):
            print('Missing saved Bert model for level "{}"'.format(levels[i]))
            sys.exit(2)
        if run_svm and (
                not os.path.exists(os.path.join(model_dir, 'svm/svm_train_level{}_vectorizer.json'.format(levels[i])))
                and not os.path.exists(os.path.join(model_dir, 'svm/svm_train_level{}_models.json'.format(levels[i])))):
            print('Missing saved SVM models for level "{}"'.format(levels[i]))
            sys.exit(2)

    # format dataset
    _, _, df_test = split_arguments(df_arguments)

    if len(df_test) < 1:
        print('There are no arguments listed for prediction.')
        sys.exit()

    # predict with Bert model
    if run_bert:
        df_bert = create_dataframe_head(df_test['Argument ID'], model_name='Bert')
        for i in range(num_levels):
            print("===> Bert: Predicting Level %s..." % levels[i])
            result = predict_bert_model(df_test, os.path.join(model_dir, 'bert_train_level{}'.format(levels[i])),
                                        values[levels[i]])
            df_bert = pd.concat([df_bert, pd.DataFrame(result, columns=values[levels[i]])], axis=1)
        df_prediction = df_bert

    # predict with SVM
    if run_svm:
        df_svm = create_dataframe_head(df_test['Argument ID'], model_name='SVM')
        for i in range(num_levels):
            print("===> SVM: Predicting Level %s..." % levels[i])
            result = predict_svm(df_test, values[levels[i]],
                                 os.path.join(model_dir, 'svm/svm_train_level{}_vectorizer.json'.format(levels[i])),
                                 os.path.join(model_dir, 'svm/svm_train_level{}_models.json'.format(levels[i])))
            df_svm = pd.concat([df_svm, result], axis=1)

        if not run_bert:
            df_prediction = df_svm
        else:
            df_prediction = pd.concat([df_prediction, df_svm])

    # predict with 1-Baseline
    if run_one_baseline:
        df_one_baseline = create_dataframe_head(df_test['Argument ID'], model_name='1-Baseline')
        for i in range(num_levels):
            print("===> 1-Baseline: Predicting Level %s..." % levels[i])
            result = predict_one_baseline(df_test, values[levels[i]])
            df_one_baseline = pd.concat([df_one_baseline, result], axis=1)

        if not run_bert and not run_svm:
            df_prediction = df_one_baseline
        else:
            df_prediction = pd.concat([df_prediction, df_one_baseline])

    # write predictions
    print("===> Writing predictions...")
    write_tsv_dataframe(os.path.join(output_dir, 'predictions_c18_20000.tsv'), df_prediction)


if __name__ == '__main__':
    main()

===> SVM: Predicting Level 1...
===> SVM: Predicting Level 2...
===> SVM: Predicting Level 3...
===> SVM: Predicting Level 4a...
===> SVM: Predicting Level 4b...
===> Writing predictions...
